# Reading and Writing Files
I like to start going over administrative scripting with the topic of files because files are fundamental to the way a Unix system thinks about data. If the filesystem were a relational database, files would be the tables, and each line would be like a record. This is obviously not true of every file, but it is a pervasive pattern. To the system, files are not only data stored on disk. They can be anything that can do IO streaming. Devices attached to the computer show up as files, sockets can show up as files and many other things as well.

## Opening Files

In [1]:
# write some text into a file
io = open("foo.txt", "w")
println(io, "Some text concerning foo.")
close(io)

# read the text from a file
io = open("foo.txt")
read(io, String)

"Some text concerning foo.\n"

The basics of working with files in Julia are not much different from other programming languages. There is an `open` method which takes then name of the file as a string and a mode argument, and returns an `IO` instance. The modes you'll most often be using are `"r"`, `"w"` and `"a"`, for _read_, _write_ and _append_. These correspond to `<`, `>` and `>>` in the shell. `"r"` is the default. There are more mode arguments, and you can read about them in the [documentation for open](https://docs.julialang.org/en/v1/base/io-network/#Base.open). There is a `write` function for writing to files, but `print` and `println` work just as well, and they will convert any non-string arguments to a string representation before sending it to the file. The `write` function, however, can also take an array of bytes (`UInt8`, in Julia parlance) and send those to the specified stream as well.

Likewise, `read` can also return an array of bytes. In fact, this is the default behavior. This is why, in our first example, the second argument, `String` is used. Here is the result if it is omitted:

In [2]:
# return to beginning of file
seek(io, 0)

show(read(io))
close(io)

UInt8[0x53, 0x6f, 0x6d, 0x65, 0x20, 0x74, 0x65, 0x78, 0x74, 0x20, 0x63, 0x6f, 0x6e, 0x63, 0x65, 0x72, 0x6e, 0x69, 0x6e, 0x67, 0x20, 0x66, 0x6f, 0x6f, 0x2e, 0x0a]

We've also seen the `close` function so far. This cleans up the file descriptor for the system and flushes any data remaining in buffers. However, you normally won't call it yourself.

For one thing, if you want to be lazy, the file descriptor will be cleaned up when the IO object is garbage-collected, so you _can_ ignore it, espeically if you're not opening many files. However, if you are opening a lot of files and you aren't sure when the garbage collector runs, There are other ways to do it. The first one is functionally similar to a context manager in Python, if you're familiar with that, but it looks a little different.

In Python, you'd write:

```python
with open("foo.txt") as io:
    print(io.read())
```
In Julia, it's a [do block](https://docs.julialang.org/en/v1/manual/functions/#Do-Block-Syntax-for-Function-Arguments-1):

In [3]:
open("foo.txt") do io
    print(read(io, String))
end

Some text concerning foo.


Do blocks with `open` are useful because they still do the cleanup step even if an exception is thrown inside the block. However, Julia has better shortcuts than that. Many functions that would take a readable `IO` instance as their argument can take the name of the file directly instead.

In [4]:
read("foo.txt", String)

"Some text concerning foo.\n"

The do-block version is always the safest if you're doing anything inside the block besides just calling the "read" function, but it doesn't make a big difference if you're not planning on using up all your file descriptors. Now let's get rid of that file and get to the next section.

In [5]:
rm("foo.txt")
# yes, that's really how you remove a file in Julia.

## Iterating on Files

In [6]:
# setup a dummy file for this section
open("dummy.txt", "w") do io
    print(io,
        """
        The first line
        Another line
        The last line
        """)
end
print(read("dummy.txt", String))

# Note that Julia truncates lines in triple-quote strings so you can still
# use pretty indentation.

The first line
Another line
The last line


Reading a file as a chunck of text is fine, but Unix tools typically need to break files into lines and deal with them one line at a time. In Julia, there are a couple ways to do this. The first is using `readlines` to read the lines in the file into an array. Like `read`, `readlines` can take an IO object or a filename as the first argument.

In [7]:
show(readlines("dummy.txt"))

["The first line", "Another line", "The last line"]

Notice that Julia has very shell-like instincts about this. Trailing newlines are skipped
automatically, whereas this takes an extra step in any other language, even Perl, whose syntax and semantics are largely based on the shell. If you want to `keep` the trailing newlines, that's also possible, just not default.

In [8]:
show(readlines("dummy.txt", keep=true))

["The first line\n", "Another line\n", "The last line\n"]

`readline` will be fine for most files, but it's not good if you have to read a large file that can't fit in memory. A more robust way to deal with lines is lazily. That's what `eachline` is for. It takes the same kind arguments as `readlines`, but doesn't load everything into memory at once. You just loop over it and get your lines.

In [9]:
for line in eachline("dummy.txt")
    println(repr(line))
end

"The first line"
"Another line"
"The last line"


`eachline` will close the file when it reaches the end, but not if iteration is interupted. Therefore, if the loop could be broken and you're worried about running out of file descriptors, it's safer to use a do block.

In [10]:
open("dummy.txt") do io
    for line in eachline(io)
        # do something
    end
end

There are many more functions you can use with `IO` objects, but this covers the common case for administrative scripting. You can read the [documentation](https://docs.julialang.org/en/v1/base/io-network/) if you want more info. We're moving on to [command-line interfaces](2-CLI.ipynb).

_Note: the Julia standard library only deals with ASCII/UTF8 strings. You may wish to check out [JuliaStrings](https://github.com/JuliaStrings) for support for other encodings._

In [ ]:
rm("dummy.txt")